# In-and export from Object storage

In [2]:
#imports
from io import StringIO
import requests
import json

import pandas as pd
import numpy as np

Credentials

In [3]:
@hidden_cell

credentials_1= {
  "auth_url": "https://identity.open.softlayer.com",
  "project": "object_storage_effacaaa_ad08_4ee4_bd59_b2e105bc9639",
  "projectId": "e96165fa44c44a7d956507ebf4026cfb",
  "region": "dallas",
  "userId": "5002ddd00432452d8dd086e3f74ed3f1",
  "username": "admin_bdffbeee0797e72fe86a4270ec23774a835cbd4b",
  "password": "TAu!-k36VR[&xp9V",
  "domainId": "193a321481be4f73b08e76a87e7d585a",
  "container":"DSETraining101ObjectStorage",
  "domainName": "1123181",
  "role": "admin"
}


In [8]:
# create some data
random_data=pd.DataFrame(np.random.random((1000, 100)))
random_data.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.218387,0.681619,0.127457,0.176184,0.208149,0.431590,0.436492,0.346246,0.632588,0.829314,...,0.432520,0.974076,0.894935,0.410939,0.487212,0.428583,0.598402,0.706644,0.385664,0.493396
1,0.438189,0.477055,0.292263,0.737119,0.986888,0.571864,0.952132,0.589837,0.010117,0.174665,...,0.301537,0.552984,0.879647,0.380063,0.300516,0.398224,0.739755,0.462363,0.046500,0.510257
2,0.246476,0.425691,0.344394,0.302752,0.592942,0.350859,0.620692,0.516486,0.033306,0.304789,...,0.131419,0.591061,0.840573,0.664932,0.472257,0.998087,0.073214,0.984443,0.957306,0.745294
3,0.303911,0.326860,0.147415,0.208054,0.371225,0.910451,0.544563,0.394877,0.424432,0.418725,...,0.556135,0.013987,0.149688,0.604044,0.574204,0.930515,0.797487,0.785470,0.447085,0.587239
4,0.885194,0.156615,0.795850,0.208292,0.658539,0.738021,0.750869,0.233611,0.890301,0.501873,...,0.149136,0.617435,0.659965,0.096339,0.922107,0.220996,0.989596,0.604612,0.461251,0.658624


In [9]:
#usecase you have a file in Python (for example from a database) and you want to create a file in local object storage

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domainId']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print resp2
    
    
# step 1: store object in local file.
# data_train is an earlier defined pandas dataframe containing data
random_data.to_csv('random_data.csv',index=False)
#step 2: move to object storage    
put_file(credentials_1,"random_data.csv")  

<Response [201]>


In [10]:
#use case: you upload a file in the object storage and want to access it in Python.

def get_object_storage_file_with_credentials(credentials,container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domainId']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

# step 1: get file from object storage
#data_train.csv is an existing object in the Object Storage 
random_data = pd.read_csv(get_object_storage_file_with_credentials(credentials_1,'DSETraining101ObjectStorage', 'random_data.csv'))
random_data.head()




,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.218387,0.681619,0.127457,0.176184,0.208149,0.431590,0.436492,0.346246,0.632588,0.829314,...,0.432520,0.974076,0.894935,0.410939,0.487212,0.428583,0.598402,0.706644,0.385664,0.493396
1,0.438189,0.477055,0.292263,0.737119,0.986888,0.571864,0.952132,0.589837,0.010117,0.174665,...,0.301537,0.552984,0.879647,0.380063,0.300516,0.398224,0.739755,0.462363,0.046500,0.510257
2,0.246476,0.425691,0.344394,0.302752,0.592942,0.350859,0.620692,0.516486,0.033306,0.304789,...,0.131419,0.591061,0.840573,0.664932,0.472257,0.998087,0.073214,0.984443,0.957306,0.745294
3,0.303911,0.326860,0.147415,0.208054,0.371225,0.910451,0.544563,0.394877,0.424432,0.418725,...,0.556135,0.013987,0.149688,0.604044,0.574204,0.930515,0.797487,0.785470,0.447085,0.587239
4,0.885194,0.156615,0.795850,0.208292,0.658539,0.738021,0.750869,0.233611,0.890301,0.501873,...,0.149136,0.617435,0.659965,0.096339,0.922107,0.220996,0.989596,0.604612,0.461251,0.658624
